In [11]:
import scipy.linalg
import scipy.optimize
import sys
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_function as tfu
tfu = importlib.reload(tfu)
from scipy.interpolate import splprep, splev
from scipy import interpolate
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from scipy.spatial.transform import Rotation as R
import scripts.theodolite_utils as ttfu


def resection_with_2_known_points(point1, point2, measures_first_point, measures_second_point):
    station_z = ((point1[2] - measures_first_point[2]*np.cos(measures_first_point[0])) + (point2[2] - measures_second_point[2]*np.cos(measures_second_point[0])))/2
    
    # find the position of the total station using the intersection of 2 circles given by the measurements of the prism position.
    radius_1 = abs(measures_first_point[2]*np.sin(measures_first_point[0]))
    radius_2 = abs(measures_second_point[2]*np.sin(measures_second_point[0]))
    
    delta_x = point2[0] - point1[0]
    delta_y = point2[1] - point1[1]
    
    # find the y value based on the intersection of 2 circles aligned in x (same value of x)
    y = (np.power(radius_1, 2) - np.power(radius_2, 2) - np.power(point1[1], 2) + np.power(point2[1], 2))/(2*delta_y)
    
    # find the possible values of x based on the quadratic formula and the circle formula knowing y
    A = 1
    B = -2*point1[0]
    C = np.power(point1[0], 2) + np.power(y, 2) + np.power(point1[1], 2) - 2*y*point1[1] - np.power(radius_1, 2)
    
    x_1 = (-B + np.sqrt(np.power(B, 2) - 4*A*C))/2*A
    
    x_2 = (-B - np.sqrt(np.power(B, 2) - 4*A*C))/2*A    
    
    # find the yaw offset to get the measures back in the prism's frame
    theta_station__prism_1 = np.arctan2(abs(point1[0]-x_1), abs(point1[1]-y))
    if (x_1 > point1[0]):
        yaw_1 =  measures_first_point[1] - (3*np.pi/2 - theta_station__prism_1)
    else:
        yaw_1 =  measures_first_point[1] - (3*np.pi/2 + theta_station__prism_1)
        
    theta_station__prism_2 = np.arctan2(abs(point1[0]-x_2), abs(point1[1]-y))
    if (x_2 > point1[0]):
        yaw_2 =  measures_first_point[1] - (3*np.pi/2 - theta_station__prism_2)
    else:
        yaw_2 =  measures_first_point[1] - (3*np.pi/2 + theta_station__prism_2)
        
    station_pos_complete_1 = np.array([x_1, y, station_z, 0, 0, yaw_1])    
    station_pos_complete_2 = np.array([x_2, y, station_z, 0, 0, yaw_2])
    
    return station_pos_complete_1, station_pos_complete_2

In [12]:
%matplotlib notebook

pilier_D = np.array([1000, 1000, 100])
pilier_A = np.array([1000, 1000 - np.sqrt(np.power(548.578, 2) - np.power(2.374, 2)), 100-2.374])
pilier_B = np.array([1000, 1000 - np.sqrt(np.power(367.562, 2) - np.power(2.051, 2)), 100-2.051])
pilier_C = np.array([1000, 1000 - np.sqrt(np.power(185.087, 2) - np.power(1.336, 2)), 100-1.336])

print(pilier_A)
print(pilier_B)
print(pilier_C)
print(pilier_D)

# Read markers file of grand axe and show them
tfu = importlib.reload(tfu)
ttfu = importlib.reload(ttfu)

file_name = './data/total_station/20220630-1/theodolite_reference_prisms.txt'

raw_1, raw_2, raw_3, trimble_1, trimble_2, trimble_3, T_1_grand, T_2_grand, T_3_grand = tfu.read_marker_file_raw_data(file_name)

station_1_pos1, station_1_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_1[0], raw_1[1])

station_2_pos1, station_2_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_2[0], raw_2[1])

station_3_pos1, station_3_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_3[0], raw_3[1])
print("Sation 1:")
print("possible position 1: ", station_1_pos1)
print("possible position 2: ", station_1_pos2)
print("Station 2:")
print("possible position 1: ", station_2_pos1)
print("possible position 2: ", station_2_pos2)
print("Station 3:")
print("possible position 1: ", station_3_pos1)
print("possible position 2: ", station_3_pos2)

[1000.          451.42713683   97.626     ]
[1000.          632.44372235   97.949     ]
[1000.          814.91782184   98.664     ]
[1000 1000  100]
Sation 1:
possible position 1:  [1015.09295056  877.08170991   98.38319957    0.            0.
   -1.39198207]
possible position 2:  [984.90704944 877.08170991  98.38319957   0.           0.
  -1.63633589]
Station 2:
possible position 1:  [1014.29359415  877.71657836   98.68521333    0.            0.
    1.20674205]
possible position 2:  [9.85706406e+02 8.77716578e+02 9.86852133e+01 0.00000000e+00
 0.00000000e+00 9.74019992e-01]
Station 3:
possible position 1:  [ 1.01350842e+03  8.78854373e+02  9.89453450e+01  0.00000000e+00
  0.00000000e+00 -3.64192049e-01]
possible position 2:  [ 9.86491578e+02  8.78854373e+02  9.89453450e+01  0.00000000e+00
  0.00000000e+00 -5.86285911e-01]


In [13]:
tfu = importlib.reload(tfu)
# choose which line is right for each total station in this section (the correct position is the first or the 2nd found with the resection)
#TF1 = tfu.tf_from_pose_roll_pitch_yaw(station_1_pos1)
TF1 = tfu.tf_from_pose_roll_pitch_yaw(station_1_pos2)

#TF2 = tfu.tf_from_pose_roll_pitch_yaw(station_2_pos1)
TF2 = tfu.tf_from_pose_roll_pitch_yaw(station_2_pos2)

#TF3 = tfu.tf_from_pose_roll_pitch_yaw(station_3_pos1)
TF3 = tfu.tf_from_pose_roll_pitch_yaw(station_3_pos2)

tp1 = TF1@trimble_1
tp2 = TF2@trimble_2
tp3 = TF3@trimble_3
tp1 = tp1.T
tp2 = tp2.T
tp3 = tp3.T

print(tp1.shape)
error = []
for i,j,k in zip(tp1,tp2,tp3):
    dist_12 = np.linalg.norm(i-j)
    dist_13 = np.linalg.norm(i-k)
    dist_23 = np.linalg.norm(k-j)
    print(dist_12)
    print(dist_13)
    print(dist_23)
    error.append(dist_12)
    error.append(dist_13)
    error.append(dist_23)
print("Average error during the calibration: ", round(np.mean(error)*1000,3), "mm")

tfu.plot_trajectories_prism(3, tp1, tp2, tp3, TF1, TF2, TF3, 0, 0,"example.pdf",1)

(40, 4)
241.98855001790912
125.86224954859614
205.09578974585537
108.77852687740537
58.581053534996286
112.23451710815549
11.710718239918442
6.294498007669762
7.623445232418907
11.10635174008959
4.765643578815589
11.108374724260994
6.685465071975471
7.2246474057601455
12.065712839993086
13.656256588441968
14.759763967528713
23.87613476231119
32.7208108304361
18.271270361205676
37.59315199479374
53.29845897268837
27.679601752227107
56.033591598819726
75.04786555028778
37.92433378220166
74.3099207951437
98.8076186917384
49.59686132981448
93.45655109518464
107.23770385482986
54.54587269513075
93.42859298986549
110.898094400525
58.60893893773442
90.83597827657357
101.88635166356974
57.662909234677635
77.8969682642405
77.79739165685848
45.09895252862353
57.96366894013503
72.0118942872725
38.69825771294751
56.95196889538124
63.44840729513473
31.707818646020122
55.26636367764981
50.22874349899936
24.537273541058457
46.169408929729705
35.173438104994865
16.890567428457253
33.50312856322998
30.

<IPython.core.display.Javascript object>